# k-Nearest Neighbour Classification

As discussed in class, the k-Nearest Neighbours method works by exploiting an existing database of _labelled_ observations. To predict the (unknown) value of a new observation, we embed it in the space of the existing observations measure the distance between the new instance and the existing observations, and then use the labels of the $k$ nearest observations to determine the prediction. In the context of classification, the prediction is the majority label found within the labels of the $k$ nearest neighbours.

To work with k-Nearest Neighbours in Python, we an make use of the existing [numpy](https://numpy.org/) and [scikit-learn](https://scikit-learn.org/) libraries. Let's start by importing them (and a [matplotlib](https://matplotlib.org/) for some plotting of the results):

In [ ]:
import numpy as np

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score

from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap

## Our "Training" Data (this is identical to Lecture 5)

We'll also need some data to act as the historical observations from our problem. For this example, we'll just make some up (but usually, you'd be given this data). In this case, we (the people doing the work) actually know the underlying function $f(X)$, but from the modelling perspective this function is not known:

In [ ]:
## This is the true underlying "generating" function of our problem, in this case
## we are using it to separate our data into two classes "a" and "b" depending
## upon where in a 2-D space an instance sits.
##
## AS WITH REGRESSION, LET'S PRETEND THAT WE DON'T KNOW THIS ONE :)
def f(X):
    t = 0.444*(X[:, 0] + 0.5)**2 + 0.5*np.sin(np.pi * X[:, 0])
    return np.where(X[:, 1] > t, 1, 0)
class_labels = np.array([ 'a', 'b' ])

Now, we will use this function to generate some training data:

In [ ]:
rng = np.random.default_rng(1234) ## notice the fixed seed for reproducability

n_points = 50
X_train = rng.uniform(-1, 1, size=(n_points, 2))
y_train = f(X_train)
X_train += rng.normal(0, 0.05, size=X_train.shape) ## let's just add a little noise to our data to make it interesting

## we'll also generate some "test" data use this to test the shape of our learned function shortly
n_test = 50
xx1, xx2 = np.meshgrid(np.linspace(-1.1, 1.1, n_test), np.linspace(-1.1, 1.1, n_test))
X = np.c_[xx1.ravel(), xx2.ravel()]
y = f(X)
Z = y.reshape(xx1.shape)

Let's take a look at the data (and underlying generating function) before moving on to modelling:

In [ ]:
a = np.linspace(-1, 1, 500)
t = 0.444*(a + 0.5)**2 + 0.5*np.sin(np.pi * a)
plt.scatter(X_train[:, 0], X_train[:, 1], color=np.where(y_train==1, 'orange', 'cornflowerblue'))
plt.plot(a, t, color='black', label='True Underlying Class Separator - f(X)')
plt.title('Our Sampled Training Data')
plt.xlabel('x1')
plt.ylabel('x2')
plt.show()

Note that the black line ($f(X)$) serves as the point of separation between two classes (although we've added a little noise to our instances, so there's a couple that manage to sneak over to opposite sides of this boundary). Remember, the function that this line represents is not known to k-Nearest Neighbours - its job is to estimate this function from available training data.

## Applying CART

Now that we have a training set of data, we can move onto modelling. We do this by instantiating a DecisionTreeClassifier model, calling the fit function, and then making predictions from the resulting model with the predict function. When building a CART model, we need to specify the minimum split size of a node (called <code>min_samples_split</code> in scikit-learn)) - here we will examine this process for a range of <code>min_samples_split</code> values:

In [ ]:
fix, ax = plt.subplots(2, 3, figsize=(16,10))
for (i, k) in enumerate([ 2, 5, 10, 15, n_points, 2*n_points ]):
    mdl = DecisionTreeClassifier(random_state=0, min_samples_split=k)
    mdl.fit(X_train, y_train)

    y_pred = mdl.predict(X)
    Z = y_pred.reshape((n_test, n_test))

    acc = accuracy_score(y, y_pred)
    r = i // 3
    c = i % 3
    ax[r, c].contourf(xx1, xx2, Z, cmap=ListedColormap(['cornflowerblue', 'orange']), alpha=0.2)
    ax[r, c].scatter(X_train[:, 0], X_train[:, 1], color=np.where(y_train == 1, 'orange', 'cornflowerblue'), label='Sampled Training Data')
    ax[r, c].plot(a, t, color='black', label='True Underlying Class Separator - f(X)')
    ax[r, c].set_title("CART Performance, minsplit={}, Accuracy={}".format(k, np.round(acc, 2)))
    ax[r, c].set_xlabel('x1')
    ax[r, c].set_ylabel('x2')
    ax[r, c].set_xlim(-1.1, 1.1)
    ax[r, c].set_ylim(-1.1, 1.1)
    ax[r, c].legend()

In each plot, the shading represents the class that would be predicted in that region of the input space. As can be seen, as <code>min_samples_split</code> increases, the complexity of the tree reduces and produces simpler (more rectangular) decision boundaries (where the class prediction changes from one label to the other).

In this case, we used a training set of 50 observations - you may wish to modify the code above so that a larger traning set is used (this can be done by modifying the n_points variable to be larger, say 250 observations). Try this and note the impact that this has on estimating the class boundaries.

## Visualising the CART model
In the above examples, CART was run on the same data six times with different values for <code>min_samples_split</code>, which resulted in six different trees being produced. One of the most valuable aspect of CART is that, for reasonably small trees, the model is highly interpretable (i.e., we can read the tree as a series of rules, and we can visualise the tree as a hierarchical series of splitting rules. To visualise a tree, we can use the <code>plot_tree</code> function from scikit-learn. For example, the six models created in the previous cell can be visualised as follows:

In [ ]:
fix, ax = plt.subplots(2, 3, figsize=(16, 10))
for (i, k) in enumerate([ 2, 5, 10, 15, n_points, 2*n_points ]):
    mdl = DecisionTreeClassifier(random_state=0, min_samples_split=k)
    mdl.fit(X_train, y_train)
    acc = accuracy_score(f(X), mdl.predict(X))

    r = i // 3
    c = i % 3
    plot_tree(mdl, ax=ax[r, c], feature_names=[ 'X1', 'X2' ], rounded=True, label='none', proportion=False, impurity=False, filled=True)
    ax[r, c].set_title("CART Performance, minsplit={}, accuracy={}".format(k, np.round(acc, 2)))

We interpret the trees as follows: the leaf nodes show the number of training instances that matched the splitting criteria up to this point (top number), along with the mean of the response of these training instances (the bottom number). The interior nodes show three things: the splitting rule at this point (top row), the number of matching training instances at this point in the tree (second row), and the mean of the response of these training instances (bottom row). We read the tree as a series of if-then-else rules. For example, the CART tree resulting from <code>min_samples_split</code>=5 can be read as follows:
<pre>
if X1 <= 0.211 then
    if X2 <= -0.034 then
        if X2 <= -0.461 then
            prediction = Class 1 (confidence = 9/9)
        else
            prediction = Class 1 (confidence = 2/4)
    else
        prediction = Class 2 (confidence = 14/14)
else
    prediction = Class 1 (confidence = 23/23)
</pre>

Note that, as <code>min_samples_split</code> _increases_, the tree complexity _decreases_, right up to the point where the tree collapses into a single node.

## Examining the effect of <code>min_samples_split</code>
The main hyperparameter for CART is the minimum node split size <code>min_samples_split</code>. In the previous step, we looked at an arbitrary set of possible values for <code>min_samples_split</code> - let's now be a little more rigorous and examine the performance of CART over a more thorough sweep of values of <code>min_samples_split</code>:

In [ ]:
all_split = np.arange(2, n_points + 1)
split_loss = []
for k in all_split:
    mdl = DecisionTreeClassifier(random_state=0, min_samples_split=k)
    mdl.fit(X_train, y_train)
    split_loss.append(accuracy_score(f(X), mdl.predict(X)))

Finally, we can plot the loss against the neighbourhood size to see how $k$ influences the behaviour of the algorithm and the resulting model:

In [ ]:
best_split = np.argmax(split_loss)
fig = plt.figure(figsize=(16,10))
plt.rcParams.update({'font.size': 24})
plt.plot(all_split, split_loss)
plt.scatter(all_split[best_split], split_loss[best_split], color='#ce2227', label="Best Accuracy, minsplit={}".format(all_split[best_split]))
plt.xlabel('minsplit')
plt.ylabel('Accuracy')
plt.ylim((0,1))
plt.title('CART Performance for minsplit')
plt.legend();

Note here that we are measuring accuracy, so a larger score is better (unlike in regression, where we were measuring error, so a lower score was better).

In this result, we can see that (for this sample of training data!) the best accuracy can be achieved with <code>min_samples_split</code> set to 1.

As mentioned above, you should repeat this step with a larger training set to see if there is any noteworthy change in behaviour.